In [2]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install kiwipiepy  #-> 형태소분석

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/79.5 MB ? eta -:--:--
     - -------------------------------------- 3.4/79.5 MB 18.3 MB/s eta 0:00:05
     ---- ----------------------------------- 9.2/79.5 MB 23.2 MB/s eta 0:00:04
     -------- ------------------------------ 16.8/79.5 MB 28.1 MB/s eta 0:00:03
     -------------- ------------------------ 29.9/79.5 MB 37.2 MB/s eta 0:00:02
     ----------------- --------------------- 35.4/79.5 MB 35.1 MB/s eta 0:00:02
     ------------------- ------------------- 39.8/79.5 MB 34.7 MB/s eta 0:00:02
     ---------------------- ---------------- 45.1/79.5 MB 31.9 MB/s eta 0:00:02
     ---------------------- ---------------- 46.1/79.5 MB 32.0 MB/s eta 0:00:02
     ------------------------ -------------- 49.3/79.5 MB 26.9 MB/s eta 0:00:02
     ----------------------------- --------- 60.6/79.5 MB 29.7 MB/s eta 0:00:01
     ---------------------------------- ---- 70.0/

  DEPRECATION: Building 'kiwipiepy_model' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'kiwipiepy_model'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
# D열에다 월간 검색수 PC + mobile의 값을 합쳐서 D 행에 합한 값을 나타내기

In [29]:
from openpyxl import load_workbook  #-> 엑셀에 있는 값들을 찾아와라
from openpyxl.styles import Font, Alignment
from kiwipiepy import Kiwi
from collections import Counter
import re



#-> 찾아오고자 하는 엑셀 파일이 이 JUPYTER 폴더와 같은 ROOTFILE에 있어야함
src = r"naverkeyword.xlsx"  #-> r : 순수 문자로 인식하게하는 것 -> .xlsx의 .을 문자로 인식하게 끔  
out = r"naverkeyword_result.xlsx"  

wb = load_workbook(src)  #-> 엑셀파일: workbook, 그안의 다수의 sheet
ws = wb.active #-> active: 엑셀 파일 열면 활성화되어있는 최초 sheet를 가져와


#==== 키워드 합계 & 사이즈 ====

#함수선언
def find_col_by_header(ws, keywords, header_row = 1) : # 엑셀시트에서 keywords를 가진 특정 column만 찾아오게하는 함수 
# keywords = ["연관키워드", "월간검색수 (PC)", "월간검색수 (mobile)", ..., "월평균노출 광고수"]
    
    for col in range(1, ws.max_column + 1) :
        v = ws.cell(row= header_row, column=col).value  
        if not v :
            continue 
        s = str(v).replace(" ", "")  #-> 띄어쓰기 없는 문자로 대체 ex) 월간검색수 (pc) -> 월간검색수(pc) 

        ok = True
        for kw in keywords :
            if kw.replace(" ", "") not in s : 
                ok = False
                break
            if ok : 
                return col
    return None 

wb = load_workbook(src)  #-> 엑셀파일: workbook, 그안의 다수의 sheet
ws = wb.active #-> active: 엑셀 파일 열면 활성화되어있는 최초 sheet를 가져와

#월간검색수 pc 열 값을 찾기
pc_col = (
    find_col_by_header(ws, ["월간검색수","PC"])
    or find_col_by_header(ws, ["월간검색수","피씨"]) 
    or 2
)
#월간검색수 mobile 값 찾기
mo_col = (
    find_col_by_header(ws, ["월간검색수","모바일"]) 
          or find_col_by_header(ws, ["월간검색수","Mobile"]) 
          or 3
)

#4,5번째 열 지정
total_col = 4
size_col = 5
ws.cell(row=1, column=total_col).value = "전체검색량(PC+모바일)"  #첫번째 행의 네번째열의 이름을 설정
ws.cell(row=1, column=size_col).value = "키워드규모(전체검색량)"

for c in [total_col, size_col] :
    cell = ws.cell(row=1, column=c)
    cell.font = Font(bold=True)
    cell.alignment = Alignment(horizontal="center")

# 정수값아 보이지만 문자인 엑셀의 값을 문자가 아닌 정수로 변환시키는 함수


def to_int(x) :
    if x is None : #셀이 비어있는 경우
        return 0
   #x가 정수거나 실수인경우
    if isinstance(x, (int, float)):
        return int(x)
    s = str(x).strip() 
    if s in ("<10","< 10") : 
        return 0
    s = s.replace(",", "")  #-> 안없애면 ,가 붙는 1000이상의 수는 없어짐
    try:  # try문이 오류가 났을때 앞에 모든걸 싹 없애지 말고 예외값을 적용해서 끝내라
        return int(float(s))
    except: 
        return 0
# 1,429.24 => int() -> 바로 정수로 변환 (에러)  -> int(float()) 실수로 먼저바꾸고 정수로 바꿔라 =>  1439

for r in range(2, ws.max_row + 1) :
    pc = to_int(ws.cell(row=r, column= pc_col).value)  #각 행의 해당 pc열에 한해서 값을 찾아와라
    mo = to_int(ws.cell(row=r, column= mo_col).value)  #각 행의 해당 mobile열에 한해서 값을 찾아와라
    total = pc+mo
    ws.cell(row=r, column= total_col).value =total

    if total >=3000 :
        size = "대형(3000+)"
    elif total >= 1000 :
        size = "중형(1000+)"
    else :
        size = "소형(<1000)"

    ws.cell(row=r, column= size_col).value = size # 5번째 열의 매 행마다 size의 값을 넣는다

ws.auto_filter.ref = f"A1:{ws.cell(row=1, column= max(ws.max_column, size_col)).coordinate}"


# ==== 키워드 형태소 분석 ======

fixed_category = {
    "원터치" : ("타입", "원터치"),
    "드롭" : ("타입", "드롭"),
    "클립" : ("타입", "클립"),
    "명품" : ("브랜드", "명품"),
}

brands = {"샤넬", "스톤헨지", "스와로브스키","까르띠에", "불가리","반클리프", "구찌" }

def classify_token(token: str) : 
   t = str(token).strip()
   if not t :
        return ("기타", "")
   if t in fixed_category:
        return fixed_category[t]

   if t in brands :
        return("브랜드", t)
   return ("기타","")

if "키워드_빈도" in wb.sheetnames : 
    del wb["키워드_빈도"]
ws_kw = wb.create_sheet("키워드_빈도")

keyword_col = 1
texts = [
    row[0] for row in ws.iter_rows(min_row = 2, max_col =1, values_only=True) 
    if row and row[0]
] # 2번행부터 하되 1번열까지만

all_text = " ".join(map(str, texts))
kiwi = Kiwi()
tokens = kiwi.tokenize(all_text)

noun_tags = {"NNG", "NNP", "NNB", "NR"}
noun_tokens = [t.form for t in tokens if t.tag in noun_tags]  #---> 이 결과 14k, 18k 등의 숫자가 다 제거됨

#정규표현식
regex_tokens = re.findall( r"[A-Za-z]*\d+[A-Za-z]+ |\d+[A-Za-z]*",all_text) #14k #18k #925 #k14    #"\d+" -> 1개 이상의 숫자

stopwords = {"엑스", "구매", "만족"}
valid_one = {"금","은", "동", "링", "침"}
combined = noun_tokens + regex_tokens 
filtered = [t for t in combined if (len(str(t)) > 1 or t in valid_one) and t not in stopwords ]

count = Counter(filtered)
ws_kw.append(["키워드","빈도", "카테고리", "세부범주"])
for cell in ws_kw[1] :
    cell.font = Font(bold = True)
    cell.alignment =  Alignment(horizontal= "center")
for k, freq in count.most_common() :
    cat, sub = classify_token(kw)
    ws_kw.append([kw, freq, cat, sub])


ws_kw.auto_filter.ref = f"A1:D{ws_kw.max_row}"




#["귀걸이", "진주귀걸이","진주목걸이"...] tuple("귀걸이",) -> 인덱스 ,iter, 반복문 가능 /// 값 추가,삭제,편집 불가
#tokens [{"tag" "N": "from": }, {"form": }]
#NNG = Noun Noun General : 일반명사 ex) 귀걸이,반지디자인 등
#NNP = Noun Noun proper : 고유명사 ex) 사넬, 티파니
#NNB = Noun Noun Bound : 의존명사 wx) 개,명,번,
#NR = Noun Nemeral : 수사 ex) 하나. 둘, 셋, 열


#데이터 관련 분야 -> 깃허브, 정규표현식


wb.save(out)
print("완료 :", out)



NameError: name 'kw' is not defined